In [6]:
import os
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## LLMs

In [7]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



Spice Odyssey


## Prompt Templates

In [8]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [9]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\n"La Cantina de los Sabores" (The Flavors\' Cantina)'

In [10]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"La Cantina de Sabor"'

In [11]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [12]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Spicy Szechuan Chicken
2. Hot and Sour Soup
3. Kung Pao Shrimp
4. General Tso's Tofu
5. Beef and Broccoli Stir Fry
6. Spicy Mapo Tofu
7. Dan Dan Noodles
8. Firecracker Shrimp
9. Sizzling Szechuan Beef
10. Spicy Garlic Eggplant
11. Ma Po Tofu
12. Spicy Thai Basil Chicken
13. Spicy Hunan Pork
14. Hot and Spicy Fish Curry
15. Spicy Sichuan Fried Rice.


#### Sequential Chain

In [13]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [14]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [15]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [16]:
chain({"cuisine": "Indian"})

C:\Users\HirushiharanThevendr\AppData\Local\Temp\ipykernel_10028\3052807682.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "Indian"})


{'cuisine': 'Indian',
 'restaurant_name': '\n"Spice Palace"',
 'menu_items': '\n\n1. Spicy Chicken Tikka Masala\n2. Vindaloo Lamb Curry\n3. Tandoori Chicken\n4. Vegetable Samosas\n5. Garlic Naan Bread\n6. Palak Paneer (spinach and cheese curry)\n7. Aloo Gobi (potato and cauliflower curry)\n8. Mango Lassi (mango yogurt smoothie)\n9. Chana Masala (chickpea curry)\n10. Chicken Biryani (spiced rice with chicken)\n11. Vegetable Korma\n12. Dal Makhani (creamy lentil curry)\n13. Butter Chicken\n14. Peshawari Naan (stuffed with dried fruits and nuts)\n15. Vegetable Pakoras (fried vegetable fritters)'}

## Agents

#### Wikipedia and llm-math tool

In [22]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his key roles in Tesla, SpaceX, and Twitter (which he rebranded as X). Since 2025, he has been a senior advisor to United States president Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk is the wealthiest person in the world; as of March 2025, Forbes estimates his net worth to be US$330 billion. 
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada, whose citizenship he had inherited through his mother. He graduated from the University of Pennsylvania in the U.S. before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment compa

'Elon Musk will be 52 years old in 2023.'

## Memory

In [23]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [24]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [25]:
chain.memory

In [26]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

C:\Users\HirushiharanThevendr\AppData\Local\Temp\ipykernel_10028\3140656249.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




"El Sabor Exquisito" (The Exquisite Flavor)


In [28]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [29]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [30]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\HirushiharanThevendr\AppData\Local\Temp\ipykernel_10028\3006166724.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [31]:
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia, with the West Indies winning by 17 runs. The tournament was hosted by England and consisted of 8 teams, with the first match being played between England and India. The West Indies team was led by captain Clive Lloyd and included notable players such as Viv Richards and Michael Holding. This victory marked the beginning of the West Indies' dominance in cricket, as they went on to win the next two world cups in 1979 and 1983."

In [32]:
convo.run("How much is 5+5?")

' 5+5 equals 10.'

In [33]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning West Indies team in the first cricket world cup in 1975 was Clive Lloyd.'

In [34]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia, with the West Indies winning by 17 runs. The tournament was hosted by England and consisted of 8 teams, with the first match being played between England and India. The West Indies team was led by captain Clive Lloyd and included notable players such as Viv Richards and Michael Holding. This victory marked the beginning of the West Indies' dominance in cricket, as they went on to win the next two world cups in 1979 and 1983.
Human: How much is 5+5?
AI:  5+5 equals 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning West Indies team in the first cricket world cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory

In [35]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

C:\Users\HirushiharanThevendr\AppData\Local\Temp\ipykernel_10028\2140498384.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


' The first cricket world cup was held in 1975 and was won by the West Indies. They defeated Australia in the final by 17 runs. The tournament was organized by the International Cricket Council and was held in England. Clive Lloyd was the captain of the winning West Indies team, while Ian Chappell was the captain of the Australian team. The man of the match in the final was West Indies player Viv Richards, who scored 138 runs. Is there anything else you would like to know about the first cricket world cup?'

In [36]:
convo.run("How much is 5+5?")

'  5+5 is equal to 10.'

In [37]:
convo.run("Who was the captain of the winning team?")

' I do not have enough context to accurately answer that question. Can you provide more information?'